In [2]:
import pysnmp

In [8]:
!pip install pysnmp schedule

  Obtaining dependency information for schedule from https://files.pythonhosted.org/packages/20/a7/84c96b61fd13205f2cafbe263cdb2745965974bdf3e0078f121dfeca5f02/schedule-1.2.2-py3-none-any.whl.metadata


In [7]:
from pysnmp.hlapi import *

def snmp_walk(ip, oid):
    result = []
    for (errorIndication, errorStatus, errorIndex, varBinds) in nextCmd(
        SnmpEngine(),
        CommunityData('public', mpModel=0),
        UdpTransportTarget((ip, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid)),
        lexicographicMode=False
    ):
        if errorIndication:
            print(f"Error with IP {ip}: {errorIndication}")
            break
        elif errorStatus:
            print('%s at %s' % (errorStatus.pr ettyPrint(),
                                errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
            break
        else:
            for varBind in varBinds:
                result.append((str(varBind[0]), str(varBind[1])))
    return result

# Printer IP address (replace with your printer's IP)
printer_ip = '10.6.151.11'

# Base OID for Printer-MIB
base_oid = '1.3.6.1.2.1.43'

# Perform the SNMP walk
oids = snmp_walk(printer_ip, base_oid)

# Print the OIDs and their values
for oid, value in oids:
    print(f'{oid} = {value}')


SyntaxError: invalid syntax. Perhaps you forgot a comma? (307640231.py, line 17)

In [5]:
from pysnmp.hlapi import *
import csv
from datetime import datetime

# List of printer IP addresses (replace with your printers' IPs)
printer_ips = ['10.6.151.11']

# OIDs for Ricoh IM C3000 (replace with the actual OIDs discovered)
oids = {
    'black_white': '1.3.6.1.2.1.43.10.2.1.4.1.1',
    'color': '1.3.6.1.2.1.43.10.2.1.4.1.2',
    'total': '1.3.6.1.2.1.43.10.2.1.4.1.3'
}

def get_snmp_data(ip, oid):
    iterator = getCmd(
        SnmpEngine(),
        CommunityData('public', mpModel=0),
        UdpTransportTarget((ip, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )
    errorIndication, errorStatus, errorIndex, varBinds = next(iterator)
    
    if errorIndication:
        print(f"Error with IP {ip}: {errorIndication}")
        return None
    elif errorStatus:
        print('%s at %s' % (errorStatus.prettyPrint(),
                            errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
        return None
    else:
        for varBind in varBinds:
            return int(varBind[1])
    return None

# Collect data
data = []
for ip in printer_ips:
    row = {'ip': ip}
    for key, oid in oids.items():
        row[key] = get_snmp_data(ip, oid)
    data.append(row)

# Save data to CSV
filename = f'printer_data_{datetime.now().strftime("%Y-%m-%d")}.csv'
with open(filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['ip', 'black_white', 'color', 'total'])
    writer.writeheader()
    writer.writerows(data)

print(f'Data saved to {filename}')


Error with IP 192.168.1.2: No SNMP response received before timeout
Error with IP 192.168.1.2: No SNMP response received before timeout
Error with IP 192.168.1.2: No SNMP response received before timeout
Error with IP 192.168.1.3: No SNMP response received before timeout
Error with IP 192.168.1.3: No SNMP response received before timeout
Error with IP 192.168.1.3: No SNMP response received before timeout
Error with IP 192.168.1.4: No SNMP response received before timeout
Error with IP 192.168.1.4: No SNMP response received before timeout
Error with IP 192.168.1.4: No SNMP response received before timeout
Data saved to printer_data_2024-07-01.csv
